In [2]:
import numpy as np
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

In [3]:
strike = 100 # Strike Price Option
s0 = 100 # Stock Price at beg
u = 1.1 # Up Factor
d= 1/u # Down Factor 
r= 0.06  # Risk Free Rate
n = 3  #no of time steps
t = 1  #time to maturity
option_type = 'C'  #Call Option

In [4]:
#For Loop With Slow tree
@timing
def binomial_Price_slow(strike,s0,u,d,r,n,t,option_type):
    #initialize Constants
    dt = t/n
    q = (np.exp(r*dt) - d)/(u-d)  # Risk Neutral upside Probablity
    disc = np.exp(-r*dt)  #discounting factor

    #initializing prices at maturity
    S = np.zeros(n+1)
    S[0] = s0*(d**n)   # Botton Most node at maturity -->  Eg we start with 100 and then at t= 3 we get the node as 100*d*d*d 

    for j in range(1,n+1):
        S[j] = S[j-1]*(u/d)    # zig zag movement from bottom most node  by divind by q we go left node and multiply by u we go right just above it

    #intitalizing call option values 
    C = np.zeros(n+1)
    for j in range(0,n+1):
        C[j] = max( S[j] - strike , 0 )   # call value = Spot - Strike 

    # going back the tree to calcuate value 
    for i in np.arange(n,0,-1):
        for j in range(0,i):
            C[j] = disc * ( (q*C[j+1] + (1-q)*C[j]))     # calulating option value at previous node for each iteration 
   
    return C[0]

binomial_Price_slow(strike,s0,u,d,r,n,t,option_type)

func:'binomial_Price_slow' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 3, 1, 'C'), {}] took: 0.0010 sec


np.float64(10.145735799928826)

In [5]:
@timing
def binomial_Price_fast(strike,s0,u,d,r,n,t,option_type):
   #initialize Constants
    dt = t/n
    q = (np.exp(r*dt) - d)/(u-d)  # Risk Neutral upside Probablity
    disc = np.exp(-r*dt)  #discounting factor

    #initializing prices at maturity
    C = s0*(d**(np.arange(n,-1,-1))) * ( u ** (np.arange(0,n+1,1)))

    #intitalizing call option values 
    C = np.maximum( C - strike , np.zeros(n+1))
    
    # going back the tree to calcuate value 
    for i in np.arange(n,0,-1):
        for j in range(0,i):
            C[j] = disc * ( (q*C[j+1] + (1-q)*C[j]))     # calulating option value at previous node for each iteration 
   
    return C[0]
    
binomial_Price_fast(strike,s0,u,d,r,n,t,option_type)

func:'binomial_Price_fast' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 3, 1, 'C'), {}] took: 0.0010 sec


np.float64(10.145735799928826)

In [7]:
for n in [10,50,100]:
    print(binomial_Price_slow(strike,s0,u,d,r,n,t,option_type))
    print(binomial_Price_fast(strike,s0,u,d,r,n,t,option_type))
  
 

func:'binomial_Price_slow' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 10, 1, 'C'), {}] took: 0.0000 sec
14.477538103234053
func:'binomial_Price_fast' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 10, 1, 'C'), {}] took: 0.0000 sec
14.477538103234021
func:'binomial_Price_slow' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 50, 1, 'C'), {}] took: 0.0030 sec
28.49926835944151
func:'binomial_Price_fast' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 50, 1, 'C'), {}] took: 0.0020 sec
28.49926835944145
func:'binomial_Price_slow' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 100, 1, 'C'), {}] took: 0.0100 sec
38.45357509478938
func:'binomial_Price_fast' args:[(100, 100, 1.1, 0.9090909090909091, 0.06, 100, 1, 'C'), {}] took: 0.0120 sec
38.45357509478929
